In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

In [2]:
models_dir = "models"
os.makedirs(models_dir, exist_ok=True)

In [3]:
df = pd.read_csv("Crop_recommendation.csv")
df

N     P     K  temperature   humidity        ph    rainfall   label
0      90.0  42.0  43.0    20.879744  82.002744  6.502985  202.935536    rice
1      85.0  58.0  41.0    21.770462  80.319644  7.038096  226.655537    rice
2      60.0  55.0  44.0    23.004459  82.320763  7.840207  263.964248    rice
3      74.0  35.0  40.0    26.491096  80.158363  6.980401  242.864034    rice
4      78.0  42.0  42.0    20.130175  81.604873  7.628473  262.717340    rice
...     ...   ...   ...          ...        ...       ...         ...     ...
2195  107.0  34.0  32.0    26.774637  66.413269       NaN  177.774507  coffee
2196   99.0  15.0  27.0    27.417112  56.636362  6.086922  127.924610  coffee
2197  118.0  33.0  30.0    24.131797  67.225123  6.362608  173.322839  coffee
2198  117.0  32.0  34.0    26.272418  52.127394  6.758793  127.175293  coffee
2199  104.0  18.0  30.0    23.603016  60.396475  6.779833  140.937041  coffee

[2200 rows x 8 columns]

In [4]:
df.rename(columns={"label": "crop"}, inplace=True)

In [5]:
df.fillna(df.mean(numeric_only=True), inplace=True)

In [6]:
label_encoder = LabelEncoder()
df["crop"] = label_encoder.fit_transform(df["crop"])
joblib.dump(label_encoder, f"{models_dir}/labelencoder.pkl")

['models/labelencoder.pkl']

In [7]:
X = df.drop(columns=["crop"])
y = df["crop"]

In [8]:
X

N     P     K  temperature   humidity        ph    rainfall
0      90.0  42.0  43.0    20.879744  82.002744  6.502985  202.935536
1      85.0  58.0  41.0    21.770462  80.319644  7.038096  226.655537
2      60.0  55.0  44.0    23.004459  82.320763  7.840207  263.964248
3      74.0  35.0  40.0    26.491096  80.158363  6.980401  242.864034
4      78.0  42.0  42.0    20.130175  81.604873  7.628473  262.717340
...     ...   ...   ...          ...        ...       ...         ...
2195  107.0  34.0  32.0    26.774637  66.413269  6.470616  177.774507
2196   99.0  15.0  27.0    27.417112  56.636362  6.086922  127.924610
2197  118.0  33.0  30.0    24.131797  67.225123  6.362608  173.322839
2198  117.0  32.0  34.0    26.272418  52.127394  6.758793  127.175293
2199  104.0  18.0  30.0    23.603016  60.396475  6.779833  140.937041

[2200 rows x 7 columns]

In [9]:
y

0       20
1       20
2       20
3       20
4       20
        ..
2195     5
2196     5
2197     5
2198     5
2199     5
Name: crop, Length: 2200, dtype: int32

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=200),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

In [12]:
best_model = None
best_accuracy = 0
best_model_name = ""
model_results = {}

In [13]:
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted', zero_division=1)
    rec = recall_score(y_test, y_pred, average='weighted', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='weighted')

    model_results[name] = {"Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1}

    print(f"\n🔹 {name} Performance:")
    print(f"   ✅ Accuracy: {acc:.4f}")
    print(f"   ✅ Precision: {prec:.4f}")
    print(f"   ✅ Recall: {rec:.4f}")
    print(f"   ✅ F1-Score: {f1:.4f}")

    if acc > best_accuracy:
        best_accuracy = acc
        best_model = model
        best_model_name = name

C:\Users\geeth\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



🔹 Logistic Regression Performance:
   ✅ Accuracy: 0.9455
   ✅ Precision: 0.9473
   ✅ Recall: 0.9455
   ✅ F1-Score: 0.9454

🔹 KNN Performance:
   ✅ Accuracy: 0.9682
   ✅ Precision: 0.9713
   ✅ Recall: 0.9682
   ✅ F1-Score: 0.9676

🔹 Random Forest Performance:
   ✅ Accuracy: 0.9886
   ✅ Precision: 0.9893
   ✅ Recall: 0.9886
   ✅ F1-Score: 0.9886


In [16]:
joblib.dump(best_model, f"{models_dir}/best_crop_recommendation.pkl")
print(f"Best Model: {best_model_name} with Accuracy: {best_accuracy:.4f}")

Best Model: Random Forest with Accuracy: 0.9886


In [17]:
y_pred_best = best_model.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred_best)

In [18]:
conf_matrix

array([[23,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0],
       [ 0, 21,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0],
       [ 0,  0, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 26,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 27,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         1,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 14,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  0,  0, 22,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 20,  0,  0,

In [21]:
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title(f'Confusion Matrix ({best_model_name})')
plt.show()

In [20]:
print("\n📊 Model Performance Comparison:")
comparison_df = pd.DataFrame(model_results).T
print(comparison_df)


📊 Model Performance Comparison:
                     Accuracy  Precision    Recall  F1-Score
Logistic Regression  0.945455   0.947263  0.945455  0.945432
KNN                  0.968182   0.971340  0.968182  0.967569
Random Forest        0.988636   0.989293  0.988636  0.988645
